In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from nltk.stem.snowball import SnowballStemmer
import re
from nltk.stem.snowball import SnowballStemmer
import nltk.corpus as corpus
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

In [2]:
train_df = pd.read_csv(r'C:\Users\Tejas\Downloads\Real_or_Not_NLP\nlp-getting-started\train.csv',encoding='utf-8')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [3]:
test_df = pd.read_csv(r'C:\Users\Tejas\Downloads\Real_or_Not_NLP\nlp-getting-started\test.csv',encoding='utf-8')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [4]:
words = set(corpus.words.words())
def check_words(text):
#     text_re = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())
    text_re = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() is not w.isalpha())
    return text_re

In [5]:
def get_tags(word):
    disaster_list = ['die','death','accident','kill','typhoon','crash','evacuation','flood','wildfire','forest fire','fire','earthquake','famine','disease','avalanches','landslides','tsunamis','volcan','plague','accident','hazardous','pollution','epidemic','armed','conflict','loss of life','violence','military','infection']
#     if len(text)==1:
#         text = list(text)
#     for text in text:
    word = word.lower()
#     print('word=',word)
    temp=[]
    for text in disaster_list:
#         print('text=',text)
        word = re.sub('#','',word)
#         print('text=',text)
        if re.findall(text,word):
            text=re.findall(text,word)
            text=",".join(text)
#             print(text)
            temp.append(text)
#             return text
    if temp:
        return temp
        return ",".join(temp)
    else:
        return 'NONE'

In [6]:
[get_tags(word) for word in train_df.text[:5]]

[['earthquake'],
 ['forest fire', 'fire'],
 ['evacuation'],
 ['evacuation', 'wildfire', 'fire'],
 ['wildfire', 'fire']]

In [7]:
def stemmer(text):
    # split into words
    tokens = word_tokenize(text)
    # stemming of words
    stemmer = SnowballStemmer('english')
    stemmed = [stemmer.stem(word) for word in tokens]
    return stemmed

In [8]:
train_df['hashtags'] = [re.findall(r"#(\w+)", text) for text in train_df.text]
# train_df['hashtags'] = [",".join(text) for text in train_df.hashtags]
train_df['reference'] = [get_tags(word) for word in train_df.text]
train_df['text_re'] = [re.sub(r'\bhttp[s]?://[a-zA-Z]*[0-9]*.*\b\s','',text) for text in train_df.text]
train_df['text_stemmed'] = [stemmer(text) for text in train_df.text]
# train_df['text_re'] = [check_words(text) for text in train_df.text_re]
train_df.head()

,id,keyword,location,text,target,hashtags,reference,text_re,text_stemmed
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,[earthquake],[earthquake],Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,"[our, deed, are, the, reason, of, this, #, earthquak, may, allah, forgiv, us, all]"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,[],"[forest fire, fire]",Forest fire near La Ronge Sask. Canada,"[forest, fire, near, la, rong, sask, ., canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,[],[evacuation],All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,"[all, resid, ask, to, shelter, in, place, ', are, be, notifi, by, offic, ., no, other, evacu, or, shelter, in, place, order, are, expect]"
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,[wildfires],"[evacuation, wildfire, fire]","13,000 people receive #wildfires evacuation orders in California","[13,000, peopl, receiv, #, wildfir, evacu, order, in, california]"
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,"[Alaska, wildfires]","[wildfire, fire]",Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,"[just, got, sent, this, photo, from, rubi, #, alaska, as, smoke, from, #, wildfir, pour, into, a, school]"


In [9]:
train_df['disaster_lvl'] = [1 if len(text)==1 else 2 if len(text)==2 else 3 for text in train_df.reference]
train_df['hashtags'] = [",".join(text) for text in train_df.hashtags]
train_df['reference'] = [",".join(text) for text in train_df.reference]
train_df['text_stemmed'] = [" ".join(text) for text in train_df.text_stemmed]
train_df.head()

,id,keyword,location,text,target,hashtags,reference,text_re,text_stemmed,disaster_lvl
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,earthquake,earthquake,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,our deed are the reason of this # earthquak may allah forgiv us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,,"forest fire,fire",Forest fire near La Ronge Sask. Canada,forest fire near la rong sask . canada,2
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,,evacuation,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,all resid ask to shelter in place ' are be notifi by offic . no other evacu or shelter in place order are expect,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,wildfires,"evacuation,wildfire,fire","13,000 people receive #wildfires evacuation orders in California","13,000 peopl receiv # wildfir evacu order in california",3
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,"Alaska,wildfires","wildfire,fire",Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,just got sent this photo from rubi # alaska as smoke from # wildfir pour into a school,2


In [10]:
# x = train_df.loc[:,['keyword','location','text','text_re']]
x = train_df.text_re
y = train_df.target

In [11]:
x.head()

0                                                                    Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
1                                                                                                   Forest fire near La Ronge Sask. Canada
2    All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
3                                                                        13,000 people receive #wildfires evacuation orders in California 
4                                                 Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school 
Name: text_re, dtype: object

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=0,stratify=y)

In [13]:
vectorizer = TfidfVectorizer()

In [14]:
x_train_transformed = vectorizer.fit_transform(x_train)
x_test_transformed  = vectorizer.transform(x_test)
feature_names = vectorizer.get_feature_names()

In [15]:
len(feature_names)

16355

In [16]:
feature_names[1000:1500]

['accept',
 'accepte',
 'accepts',
 'access',
 'accident',
 'accidentally',
 'accidents',
 'accionempresa',
 'accompanying',
 'according',
 'accordingly',
 'account',
 'accountable',
 'accuracy',
 'accused',
 'accuses',
 'accustomed',
 'acdelco',
 'ace',
 'acebreakingnews',
 'acenewsdesk',
 'acesse',
 'acfi2rhz4n',
 'achieve',
 'achievement',
 'achieving',
 'achimota',
 'aching',
 'acid',
 'acmrm833zq',
 'acoustic',
 'acousticmaloley',
 'acquiesce',
 'acquire',
 'acquired',
 'acquisitions',
 'acres',
 'acronym',
 'across',
 'acrylic',
 'act',
 'actavis',
 'acted',
 'actin',
 'acting',
 'action',
 'actionmoviestaughtus',
 'actions',
 'activate',
 'activated',
 'activates',
 'active',
 'actively',
 'activision',
 'activist',
 'activities',
 'activity',
 'actor',
 'actress',
 'acts',
 'actual',
 'actually',
 'acura',
 'acute',
 'aczruorytd',
 'ad',
 'adam',
 'adamantly',
 'adamnibloe',
 'adamrubinespn',
 'adamtuss',
 'adani',
 'adanne___',
 'add',
 'added',
 'addiction',
 'adding',
 'addi

In [17]:
x_train_transformed.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
x_train_transformed.shape

(5709, 16355)

In [19]:
x_train_transformed.toarray().shape

(5709, 16355)

In [20]:
y_train.shape,y.shape

((5709,), (7613,))

In [21]:
bernoulli_nb = BernoulliNB()
multi_nb = MultinomialNB()
gaussian_nb = GaussianNB()

In [22]:
bernoulli_nb.fit(x_train_transformed.toarray(),y_train)
multi_nb.fit(x_train_transformed.toarray(),y_train)
gaussian_nb.fit(x_train_transformed.toarray(),y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [23]:
y_predict = bernoulli_nb.predict(x_test_transformed.toarray())
print(accuracy_score(y_test, y_predict))

0.7977941176470589


In [24]:
# y_predict = multi_nb.predict(x_test_transformed.toarray())
# print(accuracy_score(y_test, y_predict))

In [25]:
# y_predict = gaussian_nb.predict(x_test_transformed.toarray())
# print(accuracy_score(y_test, y_predict))

In [26]:
scores = cross_val_score(bernoulli_nb, x_train_transformed.toarray(), y_train, cv=5, scoring="f1")
scores

array([0.77025527, 0.74712644, 0.72300469, 0.70352941, 0.75170843])

In [27]:
# scores = cross_val_score(multi_nb, x_train_transformed.toarray(), y_train, cv=5, scoring="f1")
# scores

In [28]:
# scores = cross_val_score(gaussian_nb, x_train_transformed.toarray(), y_train, cv=5, scoring="f1")
# scores

In [29]:
scores.mean() 

0.7391248487096196

In [30]:
#0.7361990040418419 without lower and check_words #bernoulli_nb
#0.7391248487096196 with removing urls #bernoulli_nb

#0.7146400456173035 with removing urls #multi_nb

#0.6323327232541376 with removing urls #gaussian_nb

In [31]:
x_transformed = vectorizer.fit_transform(x)

In [32]:
bernoulli_nb.fit(x_transformed,y)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [33]:
y_predict = bernoulli_nb.predict(x_transformed.toarray())
print(y_predict.shape)
print(accuracy_score(y, y_predict))

(7613,)
0.888480231183502


In [34]:
scores = cross_val_score(bernoulli_nb, x_transformed.toarray(), y, cv=5, scoring="f1")
scores

array([0.61074458, 0.62397373, 0.68300654, 0.64098074, 0.76234568])

In [35]:
scores.mean()

0.6642102517036159

In [36]:
train_df['model_output'] = y_predict
train_df.head()

,id,keyword,location,text,target,hashtags,reference,text_re,text_stemmed,disaster_lvl,model_output
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,earthquake,earthquake,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,our deed are the reason of this # earthquak may allah forgiv us all,1,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,,"forest fire,fire",Forest fire near La Ronge Sask. Canada,forest fire near la rong sask . canada,2,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,,evacuation,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,all resid ask to shelter in place ' are be notifi by offic . no other evacu or shelter in place order are expect,1,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,wildfires,"evacuation,wildfire,fire","13,000 people receive #wildfires evacuation orders in California","13,000 peopl receiv # wildfir evacu order in california",3,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,"Alaska,wildfires","wildfire,fire",Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,just got sent this photo from rubi # alaska as smoke from # wildfir pour into a school,2,1


In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
rf_clf = RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=5,min_samples_split=5,min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=0,
    verbose=1,
    warm_start=False,
    class_weight=None,)

In [39]:
from sklearn.preprocessing import LabelEncoder

In [40]:
enc = LabelEncoder()

In [41]:
df = train_df.copy()
df.fillna('NONE',inplace=True)
df.head()

,id,keyword,location,text,target,hashtags,reference,text_re,text_stemmed,disaster_lvl,model_output
0,1,NONE,NONE,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,earthquake,earthquake,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,our deed are the reason of this # earthquak may allah forgiv us all,1,0
1,4,NONE,NONE,Forest fire near La Ronge Sask. Canada,1,,"forest fire,fire",Forest fire near La Ronge Sask. Canada,forest fire near la rong sask . canada,2,1
2,5,NONE,NONE,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,,evacuation,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,all resid ask to shelter in place ' are be notifi by offic . no other evacu or shelter in place order are expect,1,1
3,6,NONE,NONE,"13,000 people receive #wildfires evacuation orders in California",1,wildfires,"evacuation,wildfire,fire","13,000 people receive #wildfires evacuation orders in California","13,000 peopl receiv # wildfir evacu order in california",3,1
4,7,NONE,NONE,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,"Alaska,wildfires","wildfire,fire",Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,just got sent this photo from rubi # alaska as smoke from # wildfir pour into a school,2,1


In [42]:
df.keyword = enc.fit_transform(df.keyword)
df.location = enc.fit_transform(df.location)
df.reference = enc.fit_transform(df.reference)
df.hashtags = enc.fit_transform(df.hashtags)

In [43]:
X = df.loc[:,['keyword','location','model_output','reference','hashtags','disaster_lvl']]
Y = df.target

In [44]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state=0,stratify=Y)

In [45]:
rf_clf.fit(X_train,Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=1,
                       warm_start=False)

In [46]:
Y_predict = rf_clf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [47]:
print(accuracy_score(Y_test, Y_predict))

0.8881302521008403


In [48]:
scores = cross_val_score(rf_clf, X_train, Y_train, cv=5, scoring="f1")
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

array([0.87596048, 0.85777778, 0.83636364, 0.85617978, 0.86261261])

In [49]:
scores.mean()

0.8577788570041311

In [50]:
scores = cross_val_score(rf_clf, X, Y, cv=5, scoring="f1")
print(scores)
print(scores.mean())

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

[0.60561915 0.86215539 0.85925926 0.83548665 0.60853258]
0.7542106054217733


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [51]:
df['rf_clf'] = rf_clf.predict(X)
df.head()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


,id,keyword,location,text,target,hashtags,reference,text_re,text_stemmed,disaster_lvl,model_output,rf_clf
0,1,0,1753,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,942,56,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,our deed are the reason of this # earthquak may allah forgiv us all,1,0,0
1,4,0,1753,Forest fire near La Ronge Sask. Canada,1,0,87,Forest fire near La Ronge Sask. Canada,forest fire near la rong sask . canada,2,1,1
2,5,0,1753,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,0,63,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,all resid ask to shelter in place ' are be notifi by offic . no other evacu or shelter in place order are expect,1,1,1
3,6,0,1753,"13,000 people receive #wildfires evacuation orders in California",1,1330,66,"13,000 people receive #wildfires evacuation orders in California","13,000 peopl receiv # wildfir evacu order in california",3,1,1
4,7,0,1753,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,52,112,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,just got sent this photo from rubi # alaska as smoke from # wildfir pour into a school,2,1,1


In [52]:
test_df['hashtags'] = [re.findall(r"#(\w+)", text) for text in test_df.text]
test_df['hashtags'] = [",".join(text) for text in test_df.hashtags]
test_df['reference'] = [get_tags(word) for word in test_df.text]
test_df['text_re'] = [re.sub(r'\bhttp[s]?://[a-zA-Z]*[0-9]*.*\b\s','',text) for text in test_df.text]
# test_df['text_re'] = [check_words(text) for text in test_df.text_re]
test_df['disaster_lvl'] = [1 if len(text)==1 else 2 if len(text)==2 else 3 for text in test_df.reference]
test_df['reference'] = [",".join(text) for text in test_df.reference]
test_df.head()

,id,keyword,location,text,hashtags,reference,text_re,disaster_lvl
0,0,NaN,NaN,Just happened a terrible car crash,,crash,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone.",earthquake,earthquake,"Heard about #earthquake is different cities, stay safe everyone.",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",,"forest fire,fire","there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",2
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,"Spokane,wildfires","wildfire,fire",Apocalypse lighting. #Spokane #wildfires,2
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,,"kill,typhoon",Typhoon Soudelor kills 28 in China and Taiwan,2


In [53]:
x = test_df.text_re
x_transformed = vectorizer.transform(x)
y_predict = bernoulli_nb.predict(x_transformed.toarray())
test_df['model_output'] = y_predict
test_df.head()

,id,keyword,location,text,hashtags,reference,text_re,disaster_lvl,model_output
0,0,NaN,NaN,Just happened a terrible car crash,,crash,Just happened a terrible car crash,1,0
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone.",earthquake,earthquake,"Heard about #earthquake is different cities, stay safe everyone.",1,0
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",,"forest fire,fire","there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",2,1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,"Spokane,wildfires","wildfire,fire",Apocalypse lighting. #Spokane #wildfires,2,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,,"kill,typhoon",Typhoon Soudelor kills 28 in China and Taiwan,2,1


In [54]:
df = test_df.copy()
df.fillna('NONE',inplace=True)

In [55]:
df.keyword = enc.fit_transform(df.keyword)
df.location = enc.fit_transform(df.location)
df.reference = enc.fit_transform(df.reference)
df.hashtags = enc.fit_transform(df.hashtags)

In [56]:
df.head()

,id,keyword,location,text,hashtags,reference,text_re,disaster_lvl,model_output
0,0,0,829,Just happened a terrible car crash,0,5,Just happened a terrible car crash,1,0
1,2,0,829,"Heard about #earthquake is different cities, stay safe everyone.",515,20,"Heard about #earthquake is different cities, stay safe everyone.",1,0
2,3,0,829,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",0,39,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",2,1
3,9,0,829,Apocalypse lighting. #Spokane #wildfires,380,55,Apocalypse lighting. #Spokane #wildfires,2,1
4,11,0,829,Typhoon Soudelor kills 28 in China and Taiwan,0,48,Typhoon Soudelor kills 28 in China and Taiwan,2,1


In [57]:
X = df.loc[:,['keyword','location','model_output','reference','hashtags','disaster_lvl']]

In [58]:
df['rf_clf'] = rf_clf.predict(X)
df.head()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


,id,keyword,location,text,hashtags,reference,text_re,disaster_lvl,model_output,rf_clf
0,0,0,829,Just happened a terrible car crash,0,5,Just happened a terrible car crash,1,0,0
1,2,0,829,"Heard about #earthquake is different cities, stay safe everyone.",515,20,"Heard about #earthquake is different cities, stay safe everyone.",1,0,0
2,3,0,829,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",0,39,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",2,1,1
3,9,0,829,Apocalypse lighting. #Spokane #wildfires,380,55,Apocalypse lighting. #Spokane #wildfires,2,1,1
4,11,0,829,Typhoon Soudelor kills 28 in China and Taiwan,0,48,Typhoon Soudelor kills 28 in China and Taiwan,2,1,1


In [59]:
output = pd.DataFrame(columns=['id','target'])
output.id = df.id
output.target = df.rf_clf
output.head()

,id,target
0,0,0
1,2,0
2,3,1
3,9,1
4,11,1


In [60]:
output.to_csv(r'C:\Users\Tejas\Downloads\Real_or_Not_NLP\nlp-getting-started\Real_or_Not_NLP_v2.csv',index=False)

In [61]:
output.describe()

,id,target
count,3263.000000,3263.000000
mean,5427.152927,0.311676
std,3146.427221,0.463249
min,0.000000,0.000000
25%,2683.000000,0.000000
50%,5500.000000,0.000000
75%,8176.000000,1.000000
max,10875.000000,1.000000


In [62]:
prev_output = pd.read_csv(r'C:\Users\Tejas\Downloads\Real_or_Not_NLP\nlp-getting-started\Real_or_Not_NLP.csv')
prev_output.head()

,id,target
0,0,0
1,2,0
2,3,1
3,9,1
4,11,1


In [63]:
prev_output.describe()

,id,target
count,3263.000000,3263.00000
mean,5427.152927,0.31566
std,3146.427221,0.46485
min,0.000000,0.00000
25%,2683.000000,0.00000
50%,5500.000000,0.00000
75%,8176.000000,1.00000
max,10875.000000,1.00000


In [64]:
len(output[prev_output.target==0])

2233

In [65]:
from keras.models import Sequential
from keras.layers import LSTM,Dense

Using TensorFlow backend.


In [66]:
model = Sequential()  
model.add(LSTM(100, input_shape=(x_train[1], x_train[2]),return_sequences=True))
model.add(LSTM(100,return_sequences=False))
model.add(Dense(y.shape[1],activation='Softmax'))
model.compile(loss='mean_absolute_error', optimizer='adam',metrics=['accuracy'])
model.fit(x_train, y_train, nb_epoch=10000, batch_size=1, verbose=2,validation_data=(x_test, y_test))

KeyError: 2

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [68]:
max_fatures = 30000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train_df['text'].values)
X1 = tokenizer.texts_to_sequences(train_df['text'].values)
X1 = pad_sequences(X1)
Y1 = pd.get_dummies(train_df['target']).values
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,Y1, random_state = 42)
print(X1_train.shape,Y1_train.shape)
print(X1_test.shape,Y1_test.shape)

F:\Anaconda\envs\opencv\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


(5709, 33) (5709, 2)
(1904, 33) (1904, 2)


In [69]:
embed_dim = 150
lstm_out = 200
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X1.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2,dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.


F:\Anaconda\envs\opencv\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.
F:\Anaconda\envs\opencv\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  """


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 33, 150)           4500000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               280800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 4,781,202
Trainable params: 4,781,202
Non-trainable params: 0
_________________________________________________________________
None


In [70]:
batch_size = 32
model.fit(X1_train, Y1_train, nb_epoch = 3, batch_size=batch_size, verbose = 1)

Instructions for updating:
Use tf.cast instead.


F:\Anaconda\envs\opencv\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/3


InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [71]:
score,acc = model.evaluate(X1_test, Y1_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version